# Importing libraries

In [1]:
import warnings
warnings.simplefilter(action='ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import sys
sys.path.append('/home/chayan/UMINTFS/')

import numpy as np
import pandas as pd
import tensorflow as tf
from utills import umint
from utills import datasets as ds

# Loading data and models

In [2]:
dataname = 'cbmc8k'
x1, x2, y = ds.LoadData(dataname)

In [3]:
encoder = tf.keras.models.load_model("UMINT_encoder_"+dataname+".h5", compile=False)

# Parameters

In [4]:
data = [x1, x2]
num_fs = [10,5]

# Run UMINTFS in a Supervised manner.

## Finding a cell-type specific merkers.

In [5]:
cell_type = 'B'
Marker = umint.UMINT.Find_Markers(data, y, encoder, cell_type, num_fs)
Marker

[['CD79A',
  'TCL1A',
  'CD79B',
  'HLA-DRA',
  'CD74',
  'IGHD',
  'IGHM',
  'LINC00926',
  'MS4A1',
  'HLA-DRB1'],
 ['CD45RA', 'CD19', 'CD14', 'CD16', 'CD4']]

## Finding all cell-type specific markers.

In [6]:
Markers = umint.UMINT.Find_All_Markers(data, y, encoder, num_fs)

100%|███████████████████████████████████████████| 15/15 [03:08<00:00, 12.55s/it]


# Saving the results

In [7]:
import pickle 

with open('UMINTFS_'+dataname+'_Supervised.pkl', 'wb') as f:
    pickle.dump(Markers, f)

with open('UMINTFS_'+dataname+'_Supervised.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

# Saving unique selected features list for further downstream analysis

In [8]:
markers_set = set()
for i in Markers.keys():
    markers_set = markers_set.union(set(Markers[i][0]))

In [9]:
UMINTFS_Supervised_RNA = x1[markers_set]
UMINTFS_Supervised_RNA.to_csv('UMINTFS_Supervised_'+dataname+'_rna.csv')